In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    return data

def mainBareBones():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Handle missing values for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers
    if 'Hypertension' in data.columns:
        print("Converting categorical columns to integers...")
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Initialize and train the Random Forest Classifier
    print("\nTraining the Random Forest Classifier...")
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainBareBones()

Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Training the Random Forest Classifier...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      1908



In [2]:
import pandas as pd
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    return data

def mainCuPY():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Handle only numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes to reduce memory usage
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers if necessary
    if 'Hypertension' in data.columns:
        print("Converting categorical columns to integers...")
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Convert data to CuPy arrays for GPU acceleration
    print("\nTransferring data to GPU using CuPy...")
    X_train_cp = cp.asarray(X_train, dtype=cp.float32)  # Specify dtype for efficiency
    X_test_cp = cp.asarray(X_test, dtype=cp.float32)
    y_train_cp = cp.asarray(y_train, dtype=cp.int32)
    y_test_cp = cp.asarray(y_test, dtype=cp.int32)

    # Initialize and train the Random Forest Classifier
    print("\nTraining the Random Forest Classifier...")
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    model.fit(cp.asnumpy(X_train_cp), cp.asnumpy(y_train_cp))  # Use NumPy arrays for training
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(cp.asnumpy(X_test_cp))

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(cp.asnumpy(y_test_cp), y_pred)
    print(f"Accuracy (CuPy Optimized): {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(cp.asnumpy(y_test_cp), y_pred))

if __name__ == "__main__":
    mainCuPY()


Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Transferring data to GPU using CuPy...

Training the Random Forest Classifier...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy (CuPy Optimized): 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      19

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearnex import patch_sklearn
patch_sklearn()  # Apply Intel optimizations

def optimize_datatypes(data):
    # Downcast numerical columns to reduce memory usage
    for col in data.select_dtypes(include=['int']):
        data[col] = pd.to_numeric(data[col], downcast='integer')
    for col in data.select_dtypes(include=['float']):
        data[col] = pd.to_numeric(data[col], downcast='float')
    return data

def mainIntelex():
    # Load the dataset
    file_path = "diabetes_dataset.csv"  # Replace with the path to your CSV file
    print("Loading dataset...")
    try:
        data = pd.read_csv(file_path)
        print(f"Dataset loaded successfully! Shape: {data.shape}")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit()

    # Data Cleaning and Preprocessing
    print("\nCleaning and preprocessing data...")
    missing_values_before = data.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_values_before}")
    data.fillna(data.mean(numeric_only=True), inplace=True)  # Handle missing values for numeric columns
    missing_values_after = data.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_values_after}")

    # Optimize datatypes
    print("\nOptimizing datatypes...")
    data = optimize_datatypes(data)

    # Convert categorical columns (e.g., Hypertension) to integers
    if 'Hypertension' in data.columns:
        print("Converting categorical columns to integers...")
        data['Hypertension'] = data['Hypertension'].astype(int)

    # Separate features (X) and target (y)
    print("\nSeparating features and target variable...")
    X = data.drop('Outcome', axis=1).values
    y = data['Outcome'].values
    print(f"Features shape: {X.shape}, Target shape: {y.shape}")

    # Split the data into training and testing sets
    print("\nSplitting data into training and testing sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

    # Initialize and train the Random Forest Classifier
    print("\nTraining the Random Forest Classifier...")
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)
    print("Model training completed!")

    # Make predictions
    print("\nMaking predictions on the test set...")
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("\nEvaluating the model...")
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

if __name__ == "__main__":
    mainIntelex()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Loading dataset...
Dataset loaded successfully! Shape: (9538, 17)

Cleaning and preprocessing data...
Missing values before cleaning: 0
Missing values after cleaning: 0

Optimizing datatypes...
Converting categorical columns to integers...

Separating features and target variable...
Features shape: (9538, 16), Target shape: (9538,)

Splitting data into training and testing sets...
Training set shape: (7630, 16), Testing set shape: (1908, 16)

Training the Random Forest Classifier...
Model training completed!

Making predictions on the test set...

Evaluating the model...
Accuracy: 100.00%

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1275
           1       1.00      1.00      1.00       633

    accuracy                           1.00      1908
   macro avg       1.00      1.00      1.00      1908
weighted avg       1.00      1.00      1.00      1908

